# 연관규칙분석을 통한 구매규칙 탐색(Association Rule Analysis)

## 문제 상황  

A마트는 지속적인 매출 감소가 매대 진열에 문제가 있다고 판단하였다. 따라서 매대 진열을 다시 기획하고 있다. 서로 잘 팔리는 상품에 대해서는 근접하게 매대에 배치하려고 한다. 고객의 구매 데이터(POS)를 활용해서 같이 팔리는 상품을 확인하고 이를 기반을 레이아웃을 조정하고자 한다.

## 문제 해결 프로세스  

1. 문제정의

- 지속적인 매출 감소

2. 기대효과

- 매출 증가, 회복

3. 해결방안

- 연관규칙분석을 통한 매대 레이아웃 재배치

4. 성과측정

- 매대 레이아웃 개선 전/후 매출 비교

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('C:/Users/USER/Desktop/Practice Project - ETC/연관규칙분석을 통한 구매규칙 탐색/연관규칙분석을 통한 구매규칙 탐색.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       7501 non-null   object
 1   1       5747 non-null   object
 2   2       4389 non-null   object
 3   3       3345 non-null   object
 4   4       2529 non-null   object
 5   5       1864 non-null   object
 6   6       1369 non-null   object
 7   7       981 non-null    object
 8   8       654 non-null    object
 9   9       395 non-null    object
 10  10      256 non-null    object
 11  11      154 non-null    object
 12  12      87 non-null     object
 13  13      47 non-null     object
 14  14      25 non-null     object
 15  15      8 non-null      object
 16  16      4 non-null      object
 17  17      4 non-null      object
 18  18      3 non-null      object
 19  19      1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


In [3]:
df.isnull().sum()

0        0
1     1754
2     3112
3     4156
4     4972
5     5637
6     6132
7     6520
8     6847
9     7106
10    7245
11    7347
12    7414
13    7454
14    7476
15    7493
16    7497
17    7497
18    7498
19    7500
dtype: int64

In [4]:
print("총 결측치 수: {} = 전체 데이터의 {:.2f}%".format(df.isnull().sum().sum(), (df.isnull().sum().sum()*100)/(df.shape[0]*df.shape[1])))

총 결측치 수: 120657 = 전체 데이터의 80.43%


#### 변수 해석 2가지  

1. 결측치 존재 여부

- 총 결측치의 수가 80%를 넘는 이유는 연관규칙 데이터의 특성 때문이다

- 첫번째 row는 1개의 상품을 구매한 고객들 data이고 나머지 아래 row들은 2개부터 상품을 추가로 구매한 고객들 data이다

- 따라서, 결측치는 연관규칙 분석을 위한 데이터 전처리를 통해 전부 처리할 것이다

2. 데이터 타입 설명

- object형태 20개로 구성되어 있다

3. 데이터 정보 설명

- 2개를 구매한 고객의 수는 5747명이고, 20개를 구매한 고객의 수는 1명이다

In [5]:
records = []
for i in range(len(df)):
    records.append([str(df.values[i,j]) for j in range(len(df.columns)) if not pd.isna(df.values[i,j])])

In [18]:
records

[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil'],
 ['burgers', 'meatballs', 'eggs'],
 ['chutney'],
 ['turkey', 'avocado'],
 ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea'],
 ['low fat yogurt'],
 ['whole wheat pasta', 'french fries'],
 ['soup', 'light cream', 'shallot'],
 ['frozen vegetables', 'spaghetti', 'green tea'],
 ['french fries'],
 ['eggs', 'pet food'],
 ['cookies'],
 ['turkey', 'burgers', 'mineral water', 'eggs', 'cooking oil'],
 ['spaghetti', 'champagne', 'cookies'],
 ['mineral water', 'salmon'],
 ['mineral water'],
 ['shrimp',
  'chocolate',
  'chicken',
  'honey',
  'oil',
  'cooking oil',
  'low fat yogurt'],
 ['turkey', 'eggs'],
 ['turkey',
  'fresh tuna',
  'tomatoes',
  'spagh

In [6]:
print(records[3])

['turkey', 'avocado']


i,j는 row와 col을 의미하고 (0,0)이면 shrimp를 추가해준다는 의미이다. 해당 row와 col에 shrimp와 같은 값이 있으면 list에 append 하고 결측치면 append 하지 않는다. 총 7501개의 row를 진행한다. 위의 예시는 4번째 row에 해당하는 list를 출력한 결과이며, 'turkey', 'avocado'의 값만 존재하기 때문에 2개의 값만 list에 append 한 것을 확인할 수 있다

In [7]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)

te_df = pd.DataFrame(te_ary, columns=te.columns_)

TransactionEncoder를 통해 모든 list 값을 unique하게 만든 후 컬럼으로 변경하고, 각각의 row 마다 동일한 형태로 데이터를 변경해준다. 즉, 마트에 판매하는 모든 상품의 종류를 세팅해서으로 컬럼으로 만들어주고 각각의 상품별로 구매했는지, 구매하지 않았는지를 구분 가능하게 해준다(True = 구매, False = 구매 X)

In [19]:
print("모든 상품의 종류 수 : ", len(te.columns_))

모든 상품의 종류 수 :  120


In [9]:
te_df

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


연관규칙분석을 하기위한 데이터 전처리가 완료되었다

## 구매규칙 탐색

1. 연관규칙 

- 비지도학습, 대규모 거래 데이터로부터 함께 구매될 규칙을 도출

  
- 특정 상품 구매시 이와 연관성이 높은 상품을 추천하는 것 (추천 시스템)


- 특정 사건이 발생하였을 때 함께 빈번하게 발생하는 사건의 규칙


2. Apriori

- 빈발항목집합을 통해 규칙들을 생성하는 알고리즘


- 빈발항목집합 : 최소지지도 이상을 갖는 항목집합


3. 작동원리

- 1단계 : 전체 Data set에서 빈번하게 발생하는 사건의 유형을 발견
 

- 2단계 : 최소지지도(Support) 이상을 마족하는 빈발항목 집합을 발견


- 3단계 : 빈발항목으로 집합 생성
 

- 4단계 : 새로운 빈발항목집합이 생기지 않을 때까지 2~3단계 반복 수행 
 

- 5단계 : 빈발항목 집합을 활용하여 연관규칙 생성

In [10]:
te_df.isnull().sum()

 asparagus           0
almonds              0
antioxydant juice    0
asparagus            0
avocado              0
                    ..
whole wheat pasta    0
whole wheat rice     0
yams                 0
yogurt cake          0
zucchini             0
Length: 120, dtype: int64

In [20]:
print("총 결측치 수: {} = 전체 데이터의 {:.2f}%".format(te_df.isnull().sum().sum(), (te_df.isnull().sum().sum()*100)/(df.shape[0]*df.shape[1])))

총 결측치 수: 0 = 전체 데이터의 0.00%


데이터 전처리가 완료된 데이터에는 결측치가 없는 것을 확인할 수 있다 

In [11]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemset = apriori(te_df,
                           min_support=0.005, 
                           max_len=3, 
                           use_colnames=True)

frequent_itemset['length'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)

min support = 0.5%(최소지지도 설정)

max_len = 3(리스트 개수 설정, 보통 2개 이상)

In [12]:
frequent_itemset

,support,itemsets,length
60,0.238368,(mineral water),1
27,0.179709,(eggs),1
83,0.174110,(spaghetti),1
33,0.170911,(french fries),1
20,0.163845,(chocolate),1
...,...,...,...
646,0.005066,"(tomatoes, eggs, mineral water)",3
648,0.005066,"(spaghetti, olive oil, eggs)",3
674,0.005066,"(soup, frozen vegetables, mineral water)",3
680,0.005066,"(ground beef, grated cheese, mineral water)",3


support는 전체 거래 중 약 23%가 mineral water를 구매한다는 의미이다. length 1은 단순히 어느 정도 판매되었는지만 확인하고, length 2 이상부터 의미있는 규칙을 도출할 수 있다

In [13]:
frequent_itemset[frequent_itemset['length'] >= 2].head(10)

,support,itemsets,length
503,0.059725,"(spaghetti, mineral water)",2
227,0.052660,"(chocolate, mineral water)",2
292,0.050927,"(eggs, mineral water)",2
478,0.047994,"(milk, mineral water)",2
432,0.040928,"(ground beef, mineral water)",2
236,0.039195,"(spaghetti, chocolate)",2
439,0.039195,"(ground beef, spaghetti)",2
299,0.036528,"(spaghetti, eggs)",2
277,0.036395,"(eggs, french fries)",2
387,0.035729,"(frozen vegetables, mineral water)",2


2개 이상의 Item 품목들이 있는 연관 규칙을 출력한다

In [21]:
frequent_itemset[frequent_itemset['itemsets'] == frozenset(('bacon', 'eggs'))]

,support,itemsets,length


특정 규칙을 찾고 싶을 때, 정확한 규칙을 넣어줘야한다. 'bacon'과 'eggs'를 같이 구매하는 규칙이 있을 것이라고 예측한 결과, 존재하지 않는 것을 알 수 있다

In [15]:
association_rules_df = association_rules(frequent_itemset, 
                                         metric='confidence', 
                                         min_threshold=0.005)

In [16]:
association_rules_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
2,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
3,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
4,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
...,...,...,...,...,...,...,...,...,...
1935,"(pancakes, spaghetti)",(olive oil),0.025197,0.065858,0.005066,0.201058,3.052910,0.003407,1.169224
1936,"(spaghetti, olive oil)",(pancakes),0.022930,0.095054,0.005066,0.220930,2.324260,0.002886,1.161572
1937,(pancakes),"(spaghetti, olive oil)",0.095054,0.022930,0.005066,0.053296,2.324260,0.002886,1.032075
1938,(olive oil),"(pancakes, spaghetti)",0.065858,0.025197,0.005066,0.076923,3.052910,0.003407,1.056037


상위에서 생성한 Rule에 대해, 추가 지표들에 대한 탐색을 실시한다

- antecedents support : antecedents 해당하는 상품만 구매할 확률


- consequents support : consequents 해당하는 상품만 구매할 확률


- support : antecedents 해당 상품을 구매하고, consequents 해당 상품을 구매할 확률


- confidence : antecedents 해당 상품을 구매한 고객들이 consequents 해당 상품을 구매할 확률


- lift : 단순히 antecedents 해당 상품을 구매하는 것보다 antecedents,consequents 해당 상품들을 같이 구매하는 확률이 얼마나 높은지 

In [17]:
association_rules_df[association_rules_df['consequents']==frozenset({'eggs'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158
14,(spaghetti),(eggs),0.174110,0.179709,0.036528,0.209801,1.167446,0.005239,1.038081
17,(french fries),(eggs),0.170911,0.179709,0.036395,0.212949,1.184961,0.005681,1.042232
28,(chocolate),(eggs),0.163845,0.179709,0.033196,0.202604,1.127397,0.003751,1.028711
34,(milk),(eggs),0.129583,0.179709,0.030796,0.237654,1.322437,0.007509,1.076009
...,...,...,...,...,...,...,...,...,...
1812,"(frozen vegetables, french fries)",(eggs),0.019064,0.179709,0.005199,0.272727,1.517602,0.001773,1.127900
1820,"(spaghetti, turkey)",(eggs),0.016531,0.179709,0.005199,0.314516,1.750138,0.002229,1.196659
1852,(tomato juice),(eggs),0.030396,0.179709,0.005066,0.166667,0.927423,-0.000396,0.984349
1911,"(tomatoes, mineral water)",(eggs),0.024397,0.179709,0.005066,0.207650,1.155478,0.000682,1.035263


특정 규칙을 찾고 싶을 때, 정확한 규칙을 넣어줘야한다. antecedents와 consequents를 구분해서 특정 규칙을 탐색하는 것도 가능하다

## 유의미한 구매규칙 도출 

1. Support(지지도)

- 전체 거래 항목 중 해당 규칙이 나올 확률


- "얼마나 자주 나오는 규칙인지"



2. Confidence(신뢰도)

- 조건부확률이라고도 하며, A의 거래중에서 B가 포함된 거래의 확률


- "기저귀 살때, 맥주를 같이 살 확률이 높은지"




3. Lift(향상도) 

- 임의로 B가 나올 확률 대비 A와 B가 같이 나올 확률의 비


- "자연스럽게(랜덤하게) 구매하는 확률보다 높은지"

In [22]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemset = apriori(te_df,
                           min_support=0.005, 
                           max_len=5, 
                           use_colnames=True)

frequent_itemset['length'] = frequent_itemset['itemsets'].map(lambda x: len(x))
frequent_itemset.sort_values('support',ascending=False,inplace=True)

In [23]:
frequent_itemset

,support,itemsets,length
60,0.238368,(mineral water),1
27,0.179709,(eggs),1
83,0.174110,(spaghetti),1
33,0.170911,(french fries),1
20,0.163845,(chocolate),1
...,...,...,...
646,0.005066,"(tomatoes, eggs, mineral water)",3
648,0.005066,"(spaghetti, olive oil, eggs)",3
674,0.005066,"(soup, frozen vegetables, mineral water)",3
680,0.005066,"(ground beef, grated cheese, mineral water)",3


총 725개 규칙이 있는 것을 확인할 수 있다

In [27]:
print("평균 지지도 : ", rules['support'].mean())
print()
print("평균 신뢰도 : ", rules['confidence'].mean())

평균 지지도 :  0.00895630218736482

평균 신뢰도 :  0.1577757077477294


In [25]:
rules = association_rules(frequent_itemset, metric="lift", min_threshold=1).sort_values(by = ['lift',  'confidence','support'], ascending =False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1303,(pasta),(escalope),0.015731,0.079323,0.005866,0.372881,4.700812,0.004618,1.468107
1302,(escalope),(pasta),0.079323,0.015731,0.005866,0.073950,4.700812,0.004618,1.062867
1787,(pasta),(shrimp),0.015731,0.071457,0.005066,0.322034,4.506672,0.003942,1.369601
1786,(shrimp),(pasta),0.071457,0.015731,0.005066,0.070896,4.506672,0.003942,1.059374
662,(whole wheat pasta),(olive oil),0.029463,0.065858,0.007999,0.271493,4.122410,0.006059,1.282270
...,...,...,...,...,...,...,...,...,...
1504,(escalope),(tomatoes),0.079323,0.068391,0.005466,0.068908,1.007555,0.000041,1.000555
722,(escalope),(frozen vegetables),0.079323,0.095321,0.007599,0.095798,1.005011,0.000038,1.000528
723,(frozen vegetables),(escalope),0.095321,0.079323,0.007599,0.079720,1.005011,0.000038,1.000432
1193,"(frozen vegetables, mineral water)",(french fries),0.035729,0.170911,0.006133,0.171642,1.004279,0.000026,1.000883


lift가 1이상인 유의미한 규칙만 탐색을 실시한 결과   

- 단순히 pasta만 구매하는 것보다 pasta,escalope을 같이 구매하는 확률이 약 4.7배 더 높다(lift : 4.700812)


- 전체 거래 중 pasta,escalope을 같이 구매하는 규칙이 약 0.5% 정도 된다(support : 0.005866)


- pasta를 구매한 고객들이 escalope을 구매할 확률이 약 37% 정도 된다(confidence : 0.372881)

In [28]:
rules = association_rules(frequent_itemset, metric="support", min_threshold=0.05).sort_values(by = ['support', 'lift', 'confidence'], ascending =False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
2,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
3,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
4,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
5,(mineral water),(eggs),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158


자주 발견되는 규칙을 기준으로 탐색을 실시한 결과

- 단순히 spaghetti만 구매하는 것보다 spaghetti,mineral water를 같이 구매하는 확률이 약 1.4배 더 높다(lift : 1.439085)


- 전체 거래 중 spaghetti,mineral water을 같이 구매하는 규칙이 약 5% 정도 된다(support : 0.059725)


- spaghetti를 구매한 고객들이 mineral water를 구매할 확률이 약 34% 정도 된다(confidence : 0.343032)

In [29]:
rules = association_rules(frequent_itemset, metric="confidence", min_threshold=0.3).sort_values(by = ['confidence', 'lift', 'support'], ascending =False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
258,"(soup, frozen vegetables)",(mineral water),0.007999,0.238368,0.005066,0.633333,2.656954,0.003159,2.077178
248,"(soup, olive oil)",(mineral water),0.008932,0.238368,0.005199,0.582090,2.441976,0.003070,1.822476
152,"(olive oil, frozen vegetables)",(mineral water),0.011332,0.238368,0.006532,0.576471,2.418404,0.003831,1.798297
84,"(milk, soup)",(mineral water),0.015198,0.238368,0.008532,0.561404,2.355194,0.004909,1.736520
209,"(soup, chocolate)",(mineral water),0.010132,0.238368,0.005599,0.552632,2.318395,0.003184,1.702471
...,...,...,...,...,...,...,...,...,...
134,"(green tea, chocolate)",(spaghetti),0.023464,0.174110,0.007066,0.301136,1.729574,0.002980,1.181761
35,(soup),(milk),0.050527,0.129583,0.015198,0.300792,2.321232,0.008651,1.244861
206,(mushroom cream sauce),(escalope),0.019064,0.079323,0.005733,0.300699,3.790833,0.004220,1.316568
202,"(frozen vegetables, french fries)",(milk),0.019064,0.129583,0.005733,0.300699,2.320520,0.003262,1.244697


무조건 같이 사는 규칙을 기준으로 탐색을 실시한 결과

- 단순히 (soup, frozen vegetables)만 구매하는 것보다 (soup, frozen vegetables),mineral water를 같이 구매하는 확률이 약 2.6배 더 높다(lift : 2.656954)


- 전체 거래 중 (soup, frozen vegetables),mineral water를 같이 구매하는 규칙이 약 0.5% 정도 된다(support : 0.005066)


- (soup, frozen vegetables)를 구매한 고객들이 mineral water를 구매할 확률이 약 63% 정도 된다(confidence : 0.633333)

아무리 신뢰도가 높다고 해도 향상도가 1이면 의미가 없다. 무조건 향상도가 1을 초과해야 의미있는 규칙을 도출할 수 있다. 다른 규칙들도 혼합되어서 도출될 수 있기 때문이다

성과측정은 기본 베이스 코드(위의 전체 코드)를 가지고 익월 데이터를 추가 업로드해서 성과비교가 필요하다. 예를 들어, 위의 데이터로 예측한 결과를 기반으로 매대 레이아웃 재배치를 통해 전/후 매출을 비교하는 것이다